In [ ]:
import pandas as pd
import requests
from dotenv import load_dotenv
import os
from datetime import datetime
import sqlite3
import time

In [ ]:
print(sqlite3.sqlite_version)

In [ ]:
load_dotenv(override=True)
api_key = os.getenv("HOLDERSCAN_API_KEY")
print(api_key)

In [ ]:
def get_time_stamp():
    return datetime.now().strftime('%Y%m%d%H%M%S')

In [ ]:
# directory = 'holderscan-' + get_time_stamp()
# os.makedirs(directory, exist_ok=True)

In [ ]:
base_url = 'https://api.holderscan.com/v0'
headers = {'x-api-key' : api_key}
batch_size = 100

# in seconds
request_delay = 0.25

In [ ]:
def get_holder_breakdown(ca) :
    response = requests.get(base_url + '/sol/tokens/' + ca + '/holders/breakdowns', headers=headers)
    time.sleep(request_delay)
    return response.json()

In [ ]:
def get_tokens(offset) :
    request_url = base_url + '/sol/tokens?limit=' + str(batch_size) + '&offset=' + str(offset)
    print(request_url)
    response = requests.get(request_url, headers=headers)
    time.sleep(request_delay)
    if response.status_code != 200 :
        print(response.status_code)
        print(response.json())
        return []
    return response.json()['tokens']

In [ ]:
def get_df_tokens(offset) :
    tokens = get_tokens(offset)
    if len(tokens) > 0 :
        df = pd.DataFrame(tokens)
        df['supply'] = df['supply'].astype(str)
        return df
    else :
        return pd.DataFrame()

In [ ]:
def get_tokens_total() :
    response = requests.get(base_url + '/sol/tokens', headers=headers)
    time.sleep(request_delay)
    if response.status_code == 200 :
        return response.json()['total']
    else :
        return 0

In [ ]:
db_name = 'holderscan-' + get_time_stamp() + '.db'

conn_db = sqlite3.connect(db_name)

tokens_total = get_tokens_total()
print('Total tokens: ' + str(tokens_total))

for offset_token in range(0, tokens_total, batch_size) :
    df_tokens = get_df_tokens(offset_token)
    if not df_tokens.empty :
        df_tokens.to_sql('tokens', conn_db, if_exists='append', index=False)

conn_db.close()

In [ ]:
# ca_example = '7GCihgDB8fe6KNjn2MYtkzZcRjQy3t9GHdC8uHYmW2hr'
# holder_breakdown = get_holder_breakdown(ca_example)
# print(holder_breakdown)
